In [ ]:
import torch
import open_clip
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from tqdm.notebook import tqdm
from scipy.stats import entropy
from IPython.display import display
import seaborn as sns
import os
import matplotlib.pyplot as plt
import wandb

from rtpt.rtpt import setproctitle
setproctitle('@Clipping_Privacy_CC3M_Notebook')
os.chdir('/workspace')

from datasets import FaceScrub, SingleClassSubset

%matplotlib inline

pd.set_option('display.max_rows', 15)

# Prepare the CLIP model

In [ ]:
# init clip
RUN_PATHS = {
    'top75': {'model_path': 'cc3m_experiments/checkpoints/vitb32_top75_epoch_50.pt'},
    'top50': {'model_path': 'cc3m_experiments/checkpoints/vitb32_top50_epoch_50.pt'},
    'top25': {'model_path': 'cc3m_experiments/checkpoints/vitb32_top25_epoch_50.pt'},
    'top10': {'model_path': 'cc3m_experiments/checkpoints/vitb32_top10_epoch_50.pt'},
    'top1': {'model_path': 'cc3m_experiments/checkpoints/vitb32_top01_epoch_50.pt'}
}
MODEL_NAME = 'ViT-B-32'
DATASET_NAME = 'CC2M'
device = "cuda" if torch.cuda.is_available() else "cpu"

models = {}
for num_members in RUN_PATHS.keys():
    # pretrained_model = wandb.restore(name=RUN_PATHS[num_members]['model_path'], run_path=RUN_PATHS[num_members]['run_path'])
    model, _, preprocess = open_clip.create_model_and_transforms(
        model_name=MODEL_NAME,
        pretrained=RUN_PATHS[num_members]['model_path'],
        precision='amp'
    )
    model = model.eval()
    # only append the model since the preprocessing is the same for all the models since they are all the same model type
    models[num_members] = model

In [ ]:
# define a function to get the predictions for an actor/actress
@torch.no_grad()
def get_preds_for_dataset(model, subset, context, batch_size=8, num_workers=8, device=device):
    datalaoder = DataLoader(subset, batch_size=batch_size, num_workers=num_workers, pin_memory=device == 'cuda')

    preds = []
    for x, _ in tqdm(datalaoder, desc='Iterating Dataset'):
        x = x.to(device)
        image_features, text_features, logits_scale = model(x, context)
        # we have to calculate the cosine similarity manually. OpenAI does this internally.
        logits_per_image = logits_scale  * image_features @ text_features.T
        preds.append(logits_per_image.argmax(-1).cpu())

    return torch.cat(preds)

# Load the Member and the Non-Member

In [ ]:
# load the non-members
fs_actors_non_members = pd.read_csv(
    'cc3m_experiments/conceptual_captions_facescrub_member_info/actors_non_members.csv', 
    index_col=0
).rename(columns={'name': 'class_name'})
fs_actors_non_members['name'] = fs_actors_non_members['class_name'].map(lambda x: x.replace('_', ' '))

fs_actresses_non_members = pd.read_csv(
    'cc3m_experiments/conceptual_captions_facescrub_member_info/actresses_non_members.csv', 
    index_col=0
).rename(columns={'name': 'class_name'})
fs_actresses_non_members['name'] = fs_actresses_non_members['class_name'].map(lambda x: x.replace('_', ' '))

# load the members
fs_actors_members = pd.read_csv(
    'cc3m_experiments/conceptual_captions_facescrub_member_info/actors_members.csv', 
    index_col=0
).rename(columns={'name': 'class_name'})
fs_actors_members['name'] = fs_actors_members['class_name'].map(lambda x: x.replace('_', ' '))

fs_actresses_members = pd.read_csv(
    'cc3m_experiments/conceptual_captions_facescrub_member_info/actresses_members.csv', 
    index_col=0
).rename(columns={'name': 'class_name'})
fs_actresses_members['name'] = fs_actresses_members['class_name'].map(lambda x: x.replace('_', ' '))

In [ ]:
cc_members = pd.concat([fs_actors_members, fs_actresses_members], ignore_index=True)
cc_non_members = pd.concat([fs_actors_non_members, fs_actresses_non_members], ignore_index=True)
cc_individuals = pd.concat([cc_members, cc_non_members], ignore_index=True)
cc_individuals

# Define and load the FaceScrub Dataset Class

In [ ]:
facescrub = FaceScrub(root='./data/facescrub', group='all', train=True, cropped=False, transform=preprocess)

In [ ]:
dataset_class_subsets = []
for class_idx in range(len(facescrub.classes)):
    dataset_class_subsets.append(SingleClassSubset(facescrub, class_idx))

In [ ]:
# visualize an example
plt.imshow(dataset_class_subsets[facescrub.class_to_idx[cc_members['class_name'][0]]][2][0].permute(1,2,0).numpy())
plt.show()

# Run the Model for Test Purposes on the first Actor

In [ ]:
# get the context vector of the possible labels
split_class_names = [x.replace("_", " ") for x in facescrub.classes]
label_context_vecs = open_clip.tokenize(split_class_names).to(device)

In [ ]:
test_subset_dataset = dataset_class_subsets[facescrub.class_to_idx[cc_members['class_name'][0]]]
for num_members, model in models.items():
    model = model.to(device)
    preds = get_preds_for_dataset(model, test_subset_dataset, label_context_vecs, num_workers=0)
    unique_vals, counts = preds.unique(return_counts=True)
    model = model.cpu()
    prediction = unique_vals[counts.argmax()]
    print(f'Prediction Model {num_members}: {facescrub.classes[prediction]}\t Correct Class: {facescrub.classes[test_subset_dataset.target_class]}')

In [ ]:
test_subset_dataset = dataset_class_subsets[facescrub.class_to_idx[cc_non_members['class_name'][0]]]
for num_members, model in models.items():
    model = model.to(device)
    unique_vals, counts = get_preds_for_dataset(model, test_subset_dataset, label_context_vecs).unique(return_counts=True)
    model = model.cpu()
    prediction = unique_vals[counts.argmax()]
    print(f'Prediction Model {num_members}: {facescrub.classes[prediction]}\t Correct Class: {facescrub.classes[test_subset_dataset.target_class]}')

# Run the CLIP model on all Actors
Filter for (Non-)Members afterwards using Pands

In [ ]:
concat_dataset = ConcatDataset([subset for subset in dataset_class_subsets])
preds_per_model = {}
for num_members, model in models.items():
    model = model.to(device)
    preds = get_preds_for_dataset(model, concat_dataset, label_context_vecs, batch_size=256, num_workers=64)
    model = model.cpu()
    assert len(preds) == len(concat_dataset)
    preds_per_model[num_members] = preds

In [ ]:
# split the large list of all predictions into prediction lists for every class
preds_per_model_per_subset = {}
for num_members, preds in preds_per_model.items():
    preds_per_subset = []
    counter = 0
    for subset in dataset_class_subsets:
        preds_per_subset.append(preds[counter:counter + len(subset)])
        counter += len(subset)
    preds_per_model_per_subset[num_members] = preds_per_subset

In [ ]:
preds_df_per_model = {}
for num_members in models.keys():
    df_list = []
    for group_idx, (dataset_subset, preds_subset) in enumerate(zip(dataset_class_subsets, preds_per_model_per_subset[num_members])):
        for sample_idx, pred in enumerate(preds_subset):
            class_name = facescrub.classes[dataset_class_subsets[group_idx].target_class]
            df_list.append({
                'group_idx': group_idx,
                'class_name': class_name,
                'sample_idx': sample_idx,
                'prediction': facescrub.classes[int(pred)]
            })
    preds_df = pd.DataFrame(df_list)
    preds_df_per_model[num_members] = preds_df

In [ ]:
# only get the rows of the members and non-members
for num_members, preds_df in preds_df_per_model.items():
    members = pd.merge(preds_df, cc_members['class_name'], on='class_name')
    members['actual_membership'] = 'member'
    non_members = pd.merge(preds_df, cc_non_members['class_name'], on='class_name')
    non_members['actual_membership'] = 'non_member'
    preds_df_per_model[num_members] = pd.concat([members, non_members])

In [ ]:
# TODO: uncomment this if you are running the notebook for the first time
# for num_members, preds_df in preds_df_per_model.items():
#     preds_df.to_csv(f'cc3m_experiments/prediction_dfs/predictions_{DATASET_NAME}_{MODEL_NAME}_{num_members}.csv')

preds_df_per_model = {}
for num_members, preds_df in models.items():
    preds_df_per_model[num_members] = pd.read_csv(f'cc3m_experiments/prediction_dfs/predictions_{DATASET_NAME}_{MODEL_NAME}_{num_members}.csv', index_col=0)

In [ ]:
preds_df_per_model['top75']

In [ ]:
subsample_sizes_per_model = {}
for num_members, preds_df in preds_df_per_model.items():
    min_num_images = preds_df.value_counts('class_name').sort_values()[0]
    subsample_sizes = np.arange(0, min_num_images+1, 2).tolist()
    subsample_sizes[0] = 1
    subsample_sizes_per_model[num_members] = subsample_sizes

In [ ]:
subsampled_dfs_per_model = {}
sample_draws = 20
for num_member_identities, preds_df in preds_df_per_model.items(): 
    subsample_sizes = subsample_sizes_per_model[num_member_identities]
    subsampled_dfs = []
    for sample_size in subsample_sizes:
        for i in range(sample_draws):
            membership_prediction_df = preds_df.groupby('class_name').sample(sample_size).groupby('class_name')['prediction'].agg(pd.Series.mode).apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else [x]).reset_index()
            membership_prediction_df['membership_prediction'] = membership_prediction_df.apply(lambda x: 'member' if len(x['prediction']) == 1 and x['class_name'] in x['prediction'] else 'non_member', axis='columns')
            membership_prediction_df = pd.merge(membership_prediction_df, preds_df[['class_name', 'actual_membership']].groupby('class_name')['actual_membership'].agg(pd.Series.mode), on='class_name')
            num_member, num_non_member = membership_prediction_df['actual_membership'].value_counts()['member'], membership_prediction_df['actual_membership'].value_counts()['non_member']

            tp = len(membership_prediction_df[(membership_prediction_df['membership_prediction'] == 'member') & (membership_prediction_df['actual_membership'] == 'member')])
            fp = len(membership_prediction_df[(membership_prediction_df['membership_prediction'] == 'member') & (membership_prediction_df['actual_membership'] == 'non_member')])
            fn = len(membership_prediction_df[(membership_prediction_df['membership_prediction'] == 'non_member') & (membership_prediction_df['actual_membership'] == 'member')])
            tn = len(membership_prediction_df[(membership_prediction_df['membership_prediction'] == 'non_member') & (membership_prediction_df['actual_membership'] == 'non_member')])

            subsampled_dfs.append({
                'sample_size': sample_size,
                'draw': i,
                'tpr': tp / num_member,
                'fnr': fn / num_member,
                'fpr': fp / num_non_member,
                'tnr': tn / num_non_member,
                'tp': tp,
                'fn': fn,
                'fp': fp,
                'tn': tn
            })
    subsampled_dfs_per_model[num_member_identities] = subsampled_dfs

In [ ]:
for num_members, subsampled_dfs in subsampled_dfs_per_model.items():
    subsampled_dfs_per_model[num_members] = pd.DataFrame(subsampled_dfs_per_model[num_members]).set_index('sample_size').drop('draw', axis='columns')
    subsampled_dfs_per_model[num_members] = subsampled_dfs_per_model[num_members].rename(columns={'tpr': 'True Positive Rate', 'fnr': 'False Negative Rate', 'fpr': 'False Positive Rate', 'tnr': 'True Negative Rate'})
    subsampled_dfs_per_model[num_members].index.name = 'Number of Samples'

In [ ]:
subsampled_dfs_per_model['top75']

# Evaluate the Predictions

In [ ]:
# TODO: uncomment this if you are running the notebook for the first time
# for num_members, subsampled_dfs in subsampled_dfs_per_model.items():
#     subsampled_dfs.to_csv(f'cc3m_experiments/prediction_dfs/sampled_predictions_{DATASET_NAME}_{MODEL_NAME}_{num_members}.csv')

subsampled_dfs_per_model = {}
for num_members, _ in models.items():
    subsampled_dfs_per_model[num_members] = pd.read_csv(f'cc3m_experiments/prediction_dfs/sampled_predictions_{DATASET_NAME}_{MODEL_NAME}_{num_members}.csv', index_col=0)

In [ ]:
for num_members, df in subsampled_dfs_per_model.items():
    display(df.tail(3))

In [ ]:
# calculate the accuracy for each sampling
for num_members, subsampled_dfs in subsampled_dfs_per_model.items():
    subsampled_dfs_per_model[num_members]['Accuracy'] = (subsampled_dfs['tp'] + subsampled_dfs['tn']) / (subsampled_dfs['tp'] + subsampled_dfs['tn'] + subsampled_dfs['fp'] + subsampled_dfs['fn'])

In [ ]:
sns.set_style('darkgrid')
from matplotlib.ticker import FormatStrFormatter
#for num_members, subsampled_dfs in subsampled_dfs_per_model.items():
plt.clf()
data = subsampled_dfs_per_model['top75'][['True Positive Rate', 'False Negative Rate', 'False Positive Rate', 'True Negative Rate', 'Accuracy']]
data = data.rename(columns={
    'True Positive Rate': 'TPR', 
    'False Negative Rate': 'FNR', 
    'False Positive Rate': 'FPR', 
    'True Negative Rate': 'TNR', 
    'Accuracy': 'Acc'
    }
)
ax = sns.lineplot(data=data, ci='sd')

ax.set_xlabel("Number of Attack Samples", weight="bold", size=16)
ax.set_xticklabels([int(x) for x in ax.get_xticks()], size=16)

h, l = ax.get_legend_handles_labels()
ax.set_yticklabels(ax.get_yticks(), size=16)
ax.legend(h, l, ncol=2, loc='lower center', fontsize=16)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

plt.tight_layout()
ax.get_figure().savefig(f'./cc3m_experiments/plots/subsample_plot_{DATASET_NAME}_{MODEL_NAME}_top75.pdf')
ax.get_figure().savefig(f'./cc3m_experiments/plots/subsample_plot_{DATASET_NAME}_{MODEL_NAME}_top75.png', dpi=100)
plt.show()

In [ ]:
for num_members, subsampled_dfs in subsampled_dfs_per_model.items():
    tp_std, fn_std, fp_std, tn_std = subsampled_dfs.groupby('Number of Samples').std().iloc[-1][['tp', 'fn', 'fp', 'tn']]
    tp, fn, fp, tn = subsampled_dfs.groupby('Number of Samples').mean().iloc[-1][['tp', 'fn', 'fp', 'tn']]

    tpr_std, fnr_std, fpr_std, tnr_std = subsampled_dfs.groupby('Number of Samples').std().iloc[-1][['True Positive Rate', 'False Negative Rate', 'False Positive Rate', 'True Negative Rate']]
    tpr, fnr, fpr, tnr = subsampled_dfs.groupby('Number of Samples').mean().iloc[-1][['True Positive Rate', 'False Negative Rate', 'False Positive Rate', 'True Negative Rate']]

    normalized_conf_mat = pd.DataFrame({'member': [tpr, fpr], 'non_member': [fnr, tnr]}, index=['member', 'non_member'])
    normalized_conf_mat.index.set_names('Actual Membership', inplace=True)
    normalized_conf_mat = normalized_conf_mat.rename_axis('Predicted Membership', axis='columns')

    group_names = ['TP','FN','FP','TN']
    group_counts = ["{0:0.0f} \u00B1 {1:0.2f}".format(mean, std) for mean, std in zip([tp, fn, fp, tn], [tp_std, fn_std, fp_std, tn_std])]
    percentage = ["{0:0.2f}% \u00B1 {1:0.02f}%".format(mean * 100, std * 100) for mean, std in zip([tpr, fnr, fpr, tnr], [tpr_std, fnr_std, fpr_std, tnr_std])]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names, group_counts, percentage)]
    plt.clf()
    ax = sns.heatmap(normalized_conf_mat, annot=np.asarray(labels).reshape(2, 2), fmt='', cbar=False, cmap='Blues')
    plt.tight_layout()
    ax.get_figure().savefig(f'./cc3m_experiments/plots/confusion_matrix_{DATASET_NAME}_RN50_{num_members}.pdf')
    ax.get_figure().savefig(f'./cc3m_experiments/plots/confusion_matrix_{DATASET_NAME}_RN50_{num_members}.png', dpi=100)
    print(num_members)
    plt.show()